**New York City Taxi Ride Duration Prediction**

In this case study, we will build a predictive model to predict the duration of taxi ride. We will do the following steps:
  * Install the dependencies
  * Load the data as pandas dataframe
  * Define the outcome variable - the variable we are trying to predict.
  * Build features with Deep Feature Synthesis using the [featuretools](https://featuretools.com) package. We will start with simple features and incrementally improve the feature definitions and examine the accuracy of the system.
  


Allocate at least 2-3 hours to go through this case study end-to-end

**Install Dependencies**
<p>If you have not done so already, download this repository <a href="https://github.com/Featuretools/DSx/archive/master.zip">from git</a>. Once you have downloaded this archive, unzip it and cd into the directory from the command line. Next run the command ``./install_osx.sh`` if you are on a mac or ``./install_linux.sh`` if you are on linux. This should install all of the dependencies.</p>
<p> If you are on a windows machine, open the requirements.txt folder and make sure to install each of the dependencies listed (featuretools, jupyter, pandas, sklearn, numpy) </p>
<p> Once you have installed all of the dependencies, open this notebook. On Mac and Linux, navigate to the directory that you downloaded from git and run ``jupyter notebook`` to be taken to this notebook in your default web browser. When you open the NewYorkCity_taxi_case_study.ipynb file in the web browser, you can step through the code by clicking the ``Run`` button at the top of the page. If you have any questions for how to use <a href="http://jupyter.org/">Jupyter</a>, refer to google or the discussion forum.</p>

**Running the Code**

In [2]:
import featuretools as ft
import utils
from utils import load_nyc_taxi_data, compute_features, preview, feature_importances
from sklearn.ensemble import GradientBoostingRegressor
from featuretools.primitives import (Minute, Hour, Day, Week, Month,
                                     Weekday, IsWeekend, Count, Sum, Mean, Median, Std, Min, Max)
import numpy as np
ft.__version__
%load_ext autoreload
%autoreload 2

**Step 1: Download and load the raw data as pandas dataframes**
<p>If you have not yet downloaded the data it can be downloaded <a href="https://s3.amazonaws.com/mit-dsx-data/nyc-taxi-data.zip">from S3</a>. Once you have downloaded the archive, unzip it and place the nyc-taxi-data folder in the same directory as this script. 
</p>

In [3]:
trips, pickup_neighborhoods, dropoff_neighborhoods = load_nyc_taxi_data()
preview(trips, 10)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,payment_type,trip_duration,pickup_neighborhood,dropoff_neighborhood
0,0,2,2016-01-01 00:00:19,2016-01-01 00:06:31,3,1.32,-73.961258,40.796200,-73.950050,40.787312,2,372.0,AH,C
672146,672146,1,2016-04-29 07:01:31,2016-04-29 07:15:46,1,3.30,-73.949951,40.784653,-73.982536,40.755470,1,855.0,C,AA
672147,672147,2,2016-04-29 07:01:43,2016-04-29 07:09:15,1,1.14,-73.967331,40.757370,-73.954277,40.765282,1,452.0,N,K
672148,672148,1,2016-04-29 07:01:46,2016-04-29 07:07:54,1,1.10,-74.003082,40.727509,-73.984703,40.724377,1,368.0,AB,AC
672149,672149,2,2016-04-29 07:01:46,2016-04-29 07:06:48,2,1.40,-73.990158,40.772350,-73.982147,40.759800,1,302.0,AR,AA
672150,672150,1,2016-04-29 07:01:59,2016-04-29 07:07:33,1,1.20,-73.983681,40.746677,-73.971703,40.762463,2,334.0,AO,A
672151,672151,2,2016-04-29 07:02:11,2016-04-29 07:15:24,2,2.13,-73.994209,40.750999,-73.969391,40.761539,1,793.0,D,AK
672152,672152,1,2016-04-29 07:02:11,2016-04-29 07:06:44,1,1.00,-73.983276,40.770985,-73.980110,40.760666,1,273.0,AR,A
672153,672153,2,2016-04-29 07:02:13,2016-04-29 07:08:36,1,1.17,-73.980141,40.743168,-73.983391,40.754665,1,383.0,Y,AA
672154,672154,1,2016-04-29 07:02:16,2016-04-29 07:04:07,1,0.50,-73.965973,40.765381,-73.970558,40.758724,1,111.0,AK,N


The ``trips`` table has the following fields
* ``id`` which uniquely identifies the trip
* ``vendor_id`` is the taxi cab company - in our case study we have data from three different cab companies
* ``pickup_datetime`` the time stamp for pickup
* ``dropoff_datetime`` the time stamp for drop-off
* ``passenger_count`` the number of passengers for the trip
* ``trip_distance`` total distance of the trip in miles 
* ``pickup_longitude`` the longitude for pickup
* ``pickup_latitude`` the latitude for pickup
* ``dropoff_longitude``the longitude of dropoff 
* ``dropoff_latitude`` the latitude of dropoff
* ``payment_type`` a numeric code signifying how the passenger paid for the trip. 1= Credit card 2= Cash 3= No charge 4= Dispute 5= Unknown 6= Voided
* ``trip_duration`` this is the duration we would like to predict using other fields 
* ``pickup_neighborhood`` a one or two letter id of the neighboorhood where the trip started
* ``dropoff_neighborhood`` a one or two letter id of the neighboorhood where the trip ended

**Step 2: Prepare the Data**


Lets create entities and relationships. The three entities in this data are 
* trips 
* pickup_neighborhoods
* dropoff_neighborhoods

This data has the following relationships
* pickup_neighborhoods --> trips (one neighboorhood can have multiple trips that start in it. This means pickup_neighborhoods is the ``parent_entity`` and trips is the child entity)
* dropoff_neighborhoods --> trips (one neighboorhood can have multiple trips that end in it. This means dropoff_neighborhoods is the ``parent_entity`` and trips is the child entity)

In <a <href="https://www.featuretools.com/"><featuretools (automated feature engineering software package)/></a>, we specify the list of entities and relationships as follows: 


In [4]:
entities = {
        "trips": (trips, "id", 'pickup_datetime' ),
        "pickup_neighborhoods": (pickup_neighborhoods, "neighborhood_id"),
        "dropoff_neighborhoods": (dropoff_neighborhoods, "neighborhood_id"),
        }

relationships = [("pickup_neighborhoods", "neighborhood_id", "trips", "pickup_neighborhood"),
                 ("dropoff_neighborhoods", "neighborhood_id", "trips", "dropoff_neighborhood")]

Next, we specify the cutoff time for each instance of the target_entity, in this case ``trips``.This timestamp represents the last time data can be used for calculating features by DFS. In this scenario, that would be the pickup time because we would like to make the duration prediction using data before the trip starts. 

For the purposes of the case study, we choose to only select trips that started after January 12th, 2016. 

# Question 1.1 and 1.2

**1.1 Paste in the first 5 Cutoff Times**

**1.2 Do these cutoff times make sense? Why did we choose these for the cutoff times?**

In [5]:
cutoff_time = trips[['id', 'pickup_datetime']]
cutoff_time = cutoff_time[cutoff_time['pickup_datetime'] > "2016-01-12"]
preview(cutoff_time, 10)

,id,pickup_datetime
56311,56311,2016-01-12 00:00:25
698765,698765,2016-05-03 18:54:53
698766,698766,2016-05-03 18:55:37
698767,698767,2016-05-03 18:55:38
698768,698768,2016-05-03 18:55:49
698769,698769,2016-05-03 18:55:58
698770,698770,2016-05-03 18:56:22
698771,698771,2016-05-03 18:56:24
698772,698772,2016-05-03 18:56:51
698773,698773,2016-05-03 18:56:56


**Step 3: Create baseline features using Deep Feature Synthesis**

Instead of manually creating features, such as "month of pickup datetime", we can let DFS come up with them automatically. It does this by 
* interpreting the variable types of the columns e.g categorical, numeric and others 
* matching the columns to the primitives that can be applied to their variable types
* creating features based on these matches

**Create transform features using transform primitives**

As we described in the video, features fall into two major categories, ``transform`` and ``aggregate``. In featureools, we can create transform features by specifying ``transform`` primitives. Below we specify a ``transform`` primitive called ``weekend`` and here is what it does:

* It can be applied to any ``datetime`` column in the data. 
* For each entry in the column, it assess if it is a ``weekend`` and returns a boolean. 

In this specific data, there are two ``datetime`` columns ``pickup_datetime`` and ``dropoff_datetime``. The tool automatically creates features using the primitive and these two columns as shown below. 

In [6]:
trans_primitives = [IsWeekend]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

*If you're interested about parameters to DFS such as `ignore_variables`, you can learn more about these parameters [here](https://docs.featuretools.com/generated/featuretools.dfs.html#featuretools.dfs)*
<p>Here are the features created.</p>

In [7]:
print ("Number of features: %d" % len(features))
features

Number of features: 13


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>]

Now let's compute the features. 

In [8]:
def compute_features(features, cutoff_time):
    # shuffle so we don't see encoded features in the front or backs

    np.random.shuffle(features)
    feature_matrix = ft.calculate_feature_matrix(features,
                                                 cutoff_time=cutoff_time,
                                                 approximate='36d',
                                                 verbose=True,entities=entities, relationships=relationships)
    print("Finishing computing...")
    feature_matrix, features = ft.encode_features(feature_matrix, features,
                                                  to_encode=["pickup_neighborhood", "dropoff_neighborhood"],
                                                  include_unknown=False)
    return feature_matrix

In [9]:
feature_matrix = compute_features(features, cutoff_time)

Elapsed: 00:05 | Progress: 100%|██████████
Finishing computing...


In [10]:
preview(feature_matrix, 5)

,dropoff_neighborhood = AD,dropoff_neighborhood = A,dropoff_neighborhood = AA,dropoff_neighborhood = D,dropoff_neighborhood = AR,dropoff_neighborhood = C,dropoff_neighborhood = O,dropoff_neighborhood = N,dropoff_neighborhood = AO,dropoff_neighborhood = AK,...,pickup_neighborhood = AA,pickup_neighborhood = D,pickup_neighborhood = A,pickup_neighborhood = AR,pickup_neighborhood = AK,pickup_neighborhood = AO,pickup_neighborhood = N,pickup_neighborhood = R,pickup_neighborhood = O,IS_WEEKEND(pickup_datetime)
id,,,,,,,,,,,,,,,,,,,,,
56311,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
691284,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
691285,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
691286,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,False
691288,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,False


**Build the Model**

# Question 2.1 and 2.2

**2.1 What was the Modeling Score after your last training round?**

**2.2 Hypothesize on how including more robust features will change the accuracy.**

To build a model, we
* Seperate the data into a porition for ``training`` (75% in this case) and a portion for ``testing`` 
* Get the log of the trip duration so that a more linear relationship can be found.
* Train a model using a ``GradientBoostingRegressor``

In [18]:
# separates the whole feature matrix into train data feature matrix, 
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [19]:
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train, y_train)
model.score(X_test, y_test)

      Iter       Train Loss   Remaining Time 
         1           0.4925            1.89m
         2           0.4333            1.89m
         3           0.3843            1.86m
         4           0.3446            1.88m
         5           0.3119            1.88m
         6           0.2852            1.86m
         7           0.2634            1.84m
         8           0.2454            1.81m
         9           0.2305            1.78m
        10           0.2183            1.75m
        20           0.1666            1.53m
        30           0.1558            1.30m
        40           0.1514            1.07m
        50           0.1488           51.47s
        60           0.1472           40.10s
        70           0.1458           29.31s
        80           0.1448           19.33s
        90           0.1440            9.57s
       100           0.1433            0.00s


0.7220107526801756

# Question 3.1 and 3.2

**3.1 Paste in 5 of the  generated features**

**3.2 What are these features called and how are they generated?**

**Step 5: Adding more Transform Primitives**

* Add ``Minute``, ``Hour``, ``Week``, ``Month``, ``Weekday`` , etc primitives
* All these transform primitives apply to ``datetime`` columns

In [11]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, IsWeekend]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=[],
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [12]:
print ("Number of features: %d" % len(features))
features

Number of features: 25


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>]

Now let's compute the features. 

In [13]:
feature_matrix = compute_features(features, cutoff_time)

Elapsed: 00:06 | Progress: 100%|██████████
Finishing computing...


In [14]:
preview(feature_matrix, 10)

,IS_WEEKEND(dropoff_datetime),MINUTE(pickup_datetime),HOUR(pickup_datetime),WEEKDAY(pickup_datetime),MINUTE(dropoff_datetime),HOUR(dropoff_datetime),pickup_neighborhood = AD,pickup_neighborhood = AA,pickup_neighborhood = D,pickup_neighborhood = A,...,trip_duration,DAY(pickup_datetime),trip_distance,WEEKDAY(dropoff_datetime),WEEK(pickup_datetime),pickup_neighborhoods.longitude,dropoff_neighborhoods.latitude,vendor_id,IS_WEEKEND(pickup_datetime),pickup_neighborhoods.latitude
id,,,,,,,,,,,,,,,,,,,,,
56311,False,0,0,1,11,0,0,0,0,0,...,645.0,12,1.61,1,2,-73.987205,40.721435,2,False,40.720245
691284,False,21,12,0,24,12,0,0,0,0,...,160.0,2,0.61,0,18,-73.991595,40.721435,2,False,40.729652
691285,False,22,12,0,27,12,0,0,0,0,...,295.0,2,0.88,0,18,-73.982322,40.785005,2,False,40.776270
691286,False,22,12,0,48,12,0,0,0,0,...,1573.0,2,1.90,0,18,-73.977943,40.757707,1,False,40.742531
691288,False,23,12,0,30,12,0,0,0,0,...,404.0,2,1.00,0,18,-73.985336,40.761087,1,False,40.747126
691289,False,23,12,0,55,12,0,0,0,0,...,1906.0,2,3.24,0,18,-73.998366,40.761492,2,False,40.721435
691290,False,23,12,0,26,12,0,0,0,0,...,156.0,2,0.10,0,18,-73.966696,40.764723,1,False,40.764723
691291,False,24,12,0,37,12,0,0,0,0,...,827.0,2,1.60,0,18,-73.956886,40.776270,1,False,40.766809
691292,False,24,12,0,39,12,1,0,0,0,...,883.0,2,1.50,0,18,-73.976515,40.764723,1,False,40.752186


# Question 4.1 and 4.2

**4.1 What was the Modeling Score after your last training round when including the transform primitives?**

**4.2 Comment on how the modeling accuracy differs when including the Transform features.**


**Step 6: Build the new model**

In [15]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [16]:
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train,y_train)
model.score(X_test,y_test)

      Iter       Train Loss   Remaining Time 
         1           0.4925            2.26m
         2           0.4333            2.18m
         3           0.3843            2.14m
         4           0.3444            2.10m
         5           0.3117            2.07m
         6           0.2848            2.04m
         7           0.2620            2.02m
         8           0.2435            2.01m
         9           0.2282            1.98m
        10           0.2152            1.95m
        20           0.1588            1.69m
        30           0.1415            1.40m
        40           0.1332            1.16m
        50           0.1283           55.96s
        60           0.1252           43.27s
        70           0.1227           31.51s
        80           0.1207           20.59s
        90           0.1191           10.13s
       100           0.1177            0.00s


0.7755557670208891

**Step 7: Add Aggregation Primitives**

Now let's add aggregation primitives. These primitives will generate features for the parent entities ``pickup_neighborhoods``, and ``dropoff_neighborhood`` and then add them to the trips entity, which is the entity for which we are trying to make prediction.

In [26]:
trans_primitives = [Minute, Hour, Day, Week, Month, Weekday, IsWeekend]
aggregation_primitives = [Count, Sum, Mean, Median, Std, Max, Min]

features = ft.dfs(entities=entities,
                  relationships=relationships,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=aggregation_primitives,
                  ignore_variables={"trips": ["pickup_latitude", "pickup_longitude",
                                              "dropoff_latitude", "dropoff_longitude"]},
                  features_only=True)

In [27]:
print ("Number of features: %d" % len(features))
features

Number of features: 63


[<Feature: vendor_id>,
 <Feature: passenger_count>,
 <Feature: trip_distance>,
 <Feature: payment_type>,
 <Feature: trip_duration>,
 <Feature: pickup_neighborhood>,
 <Feature: dropoff_neighborhood>,
 <Feature: MINUTE(dropoff_datetime)>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: HOUR(dropoff_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: DAY(dropoff_datetime)>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: WEEK(dropoff_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: MONTH(dropoff_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEKDAY(dropoff_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: IS_WEEKEND(dropoff_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: pickup_neighborhoods.latitude>,
 <Feature: pickup_neighborhoods.longitude>,
 <Feature: dropoff_neighborhoods.latitude>,
 <Feature: dropoff_neighborhoods.longitude>,
 <Feature: pickup_neighborhoods.COUNT(trips)>,
 <Feature: pickup_neighborhoods.SUM(trips.passenger_count)>

In [28]:
feature_matrix = compute_features(features, cutoff_time)

Elapsed: 00:19 | Progress: 100%|██████████
Finishing computing...


In [29]:
preview(feature_matrix, 10)

,dropoff_neighborhoods.MAX(trips.trip_duration),pickup_neighborhoods.MAX(trips.passenger_count),pickup_neighborhoods.MAX(trips.trip_distance),WEEKDAY(pickup_datetime),dropoff_neighborhoods.MEDIAN(trips.trip_distance),pickup_neighborhoods.longitude,trip_duration,DAY(dropoff_datetime),pickup_neighborhoods.MEAN(trips.trip_distance),pickup_neighborhood = AD,...,dropoff_neighborhoods.MIN(trips.trip_duration),dropoff_neighborhoods.MAX(trips.trip_distance),WEEK(pickup_datetime),payment_type,dropoff_neighborhoods.SUM(trips.trip_duration),HOUR(pickup_datetime),pickup_neighborhoods.MIN(trips.passenger_count),WEEK(dropoff_datetime),pickup_neighborhoods.MAX(trips.trip_duration),pickup_neighborhoods.COUNT(trips)
id,,,,,,,,,,,,,,,,,,,,,
56311,3572.0,6.0,18.57,1,1.75,-73.987205,645.0,12,2.978551,0,...,1.0,20.54,2,1,1053070.0,0,1.0,2,3292.0,1332.0
691284,3603.0,6.0,22.71,0,1.69,-73.991595,160.0,2,2.232347,0,...,1.0,24.10,18,1,13871368.0,12,1.0,18,3606.0,20658.0
691285,3602.0,6.0,26.44,0,1.60,-73.982322,295.0,2,2.062772,0,...,0.0,86.52,18,1,12276557.0,12,1.0,18,3580.0,21800.0
691286,3606.0,6.0,25.64,0,1.37,-73.977943,1573.0,2,2.125305,0,...,2.0,502.80,18,1,24003066.0,12,1.0,18,3587.0,18764.0
691288,3580.0,6.0,22.95,0,1.50,-73.985336,404.0,2,2.171776,0,...,2.0,26.62,18,1,12359849.0,12,0.0,18,3598.0,25905.0
691289,3606.0,6.0,502.80,0,1.40,-73.998366,1906.0,2,2.509054,0,...,0.0,25.60,18,1,24417513.0,12,1.0,18,3561.0,18237.0
691290,3580.0,6.0,21.60,0,1.30,-73.966696,156.0,2,1.830726,0,...,0.0,21.60,18,2,14790756.0,12,1.0,18,3604.0,26310.0
691291,3604.0,6.0,20.90,0,1.52,-73.956886,827.0,2,2.266902,0,...,1.0,501.40,18,1,14027834.0,12,1.0,18,3579.0,19798.0
691292,3580.0,6.0,32.09,0,1.30,-73.976515,883.0,2,2.274509,1,...,0.0,21.60,18,1,14790756.0,12,1.0,18,3602.0,35371.0


# Question 5.1 and 5.2

**5.1 What was the Modeling Score after your last training round when including the aggregate transforms?**

**5.2 How do these aggregate transforms impact performance? How do they impact training time?**

**Step 8: Build the new model**

In [17]:
# separates the whole feature matrix into train data feature matrix,
# train data labels, and test data feature matrix 
X_train, y_train, X_test, y_test = utils.get_train_test_fm(feature_matrix,.75)
y_train = np.log(y_train+1)
y_test = np.log(y_test+1)

In [18]:
# note: this may take up to 30 minutes to run
model = GradientBoostingRegressor(verbose=True)
model.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.4925            2.23m
         2           0.4333            2.16m
         3           0.3843            2.15m
         4           0.3444            2.11m
         5           0.3117            2.09m
         6           0.2848            2.05m
         7           0.2620            2.02m
         8           0.2435            1.99m
         9           0.2282            1.97m
        10           0.2152            1.95m
        20           0.1588            1.69m
        30           0.1415            1.41m
        40           0.1332            1.16m
        50           0.1283           56.05s
        60           0.1252           43.27s
        70           0.1227           31.42s
        80           0.1207           20.53s
        90           0.1191           10.07s
       100           0.1177            0.00s


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=True,
                          warm_start=False)

# Step 9: Evalute on test data

In [19]:
model.score(X_test,y_test)

0.7755608981558122

we can also make predictions using our model

In [20]:
y_pred = model.predict(X_test)
y_pred = np.exp(y_pred) - 1 # undo the log we took earlier
y_pred[5:]

array([ 553.50659125,  550.18452273, 1480.48337338, ..., 1106.04597122,
       1794.41849311,  789.39461652])

# Additional Analysis
<p>Let's look at how important each feature was for the model.</p>

In [21]:
feature_importances(model, feature_matrix.columns, n=15)

1: Feature: DAY(pickup_datetime), 0.894
2: Feature: HOUR(pickup_datetime), 0.027
3: Feature: HOUR(dropoff_datetime), 0.027
4: Feature: pickup_neighborhoods.longitude, 0.018
5: Feature: WEEKDAY(pickup_datetime), 0.007
6: Feature: dropoff_neighborhoods.longitude, 0.006
7: Feature: vendor_id, 0.003
8: Feature: trip_distance, 0.003
9: Feature: IS_WEEKEND(pickup_datetime), 0.003
10: Feature: WEEK(pickup_datetime), 0.002
11: Feature: dropoff_neighborhood = AA, 0.002
12: Feature: MONTH(dropoff_datetime), 0.001
13: Feature: payment_type, 0.001
14: Feature: dropoff_neighborhood = A, 0.001
15: Feature: IS_WEEKEND(dropoff_datetime), 0.001
